#### Prerequesites:
* Define schema (make figure) 
* Define structure (make figure)
* Define schema class in python (src/models/model_schemata.py)
* Convert schema + structure into model template (models/templates/p_model.pl)

#### Procedure:
* Load data
* Adjust model script for lfi accordingly
* Create evidence file (if I don't need different evidence files, move to builmodel notebook)
* LFI

Multiple instances, 1 pst, X enzymes, one sample per instance, multiple enzymes per instance, test how many enzymes can have in one instances, how computing time depends on N p::f for 60 samples

In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
# Importing libraries
import pandas as pd
import numpy as np
import os
import random

from src.models import model_schemata as schema
from src.models import build_model as build
from src.visualization import visualize as viz

In [10]:
os.chdir('/Users/magdalena/OneDrive - Queen Mary, University of London/bezzlab/research/projects/phospho_pi/')
# os.chdir('/home/mhuebner/Desktop/bezzlab/research/projects/phospho_pi/')

#### Training/Testing

In [4]:
# reading csv from files into dict
data = {}
data['e_ksea'] = pd.read_csv('data/processed/ebdt_data/sub_network_e/e_ksea.csv')
data['p_fc'] = pd.read_csv('data/processed/ebdt_data/sub_network_e/p_fc.csv')
data['p_regulates'] = pd.read_csv('data/processed/ebdt_data/sub_network_e/p_regulates.csv')
data['e_ksea']

,enzyme,sample,tc,value,prob,p_dec,p_inc
0,ABL1,AC220,10.0,inc,0.648536,0.001000,0.648536
1,ABL1,AT13148,10.0,inc,0.590795,0.001000,0.590795
2,ABL1,AZ20,10.0,inc,0.651697,0.001000,0.651697
3,ABL1,AZD1480,10.0,inc,0.569660,0.001000,0.569660
4,ABL1,AZD3759,10.0,dec,0.836416,0.836416,0.001000
...,...,...,...,...,...,...,...
361,SRC,Torin,12.0,inc,0.774683,0.001000,0.774683
362,SRC,Trametinib,12.0,inc,0.735235,0.001000,0.735235
363,SRC,U73122,12.0,inc,0.550325,0.001000,0.550325
364,SRC,Ulixertinib,12.0,inc,0.601812,0.001000,0.601812


In [40]:
data['p_fc']['phosphosite'].value_counts()

ABL1(S569)     61
ABL1(S718)     61
ABL1(T735)     61
HIPK2(Y361)    61
PTK2(S29)      61
PTK2(S722)     61
PTK2(S910)     61
PTPRG(S995)    61
SRC(S17)       61
SRC(S75)       61
PTK2(S843)     61
Name: phosphosite, dtype: int64

Splitting data into training/testing

In [46]:
# getting sample names (union of all samples in all datasets)
samples = list(set(data['e_ksea']['sample']).union(set(data['p_fc']['sample'])))
samples.sort()
# sample x% of samples randomly without replacement with seed
random.seed(612)
train = random.sample(samples, int(len(samples)*0.8))
print(train)
# filter data
training_data = {}
training_data['e_ksea'] = data['e_ksea'][data['e_ksea']['sample'].isin(train)].reset_index(drop=True)
training_data['p_fc'] = data['p_fc'][data['p_fc']['sample'].isin(train)].reset_index(drop=True)

['TAK715', 'TGX221', 'CHIR99021', 'CAL101', 'Ripasudil', 'KN93', 'LY2584702', 'LY2090314', 'Ku0063794', 'SP600125', 'Ribociclib', 'U73122', 'AZD5438', 'GDC0994', 'Edelfosine', 'PF3758309', 'Amuvatinib', 'AT13148', 'AZD3759', 'Go6976', 'JNJ', 'NU7441', 'GSK2334470', 'Ipatasertib', 'AZD5363', 'LY2835219', 'Torin', 'Bosutinib', 'H89', 'DNAPK', 'GDC0941', 'FRAX486', 'CX4945', 'HS173', 'Linsitinib', 'Dabrafenib', 'AZD1480', 'BX912', 'AZ20', 'TBCA', 'Selumetinib', 'GF109203X', 'AZD6482', 'AC220', 'PH797804', 'AZD8055', 'Trametinib', 'JNK']


In [47]:
# Mapping data to Problog predicates
predicates = {}
predicates['e_ksea'] = schema.EKseaPredicate()
predicates['e_ksea'].add_data(training_data['e_ksea'], 'enzyme', 'sample', 'value')
predicates['p_fc'] = schema.PFoldChangePredicate()
predicates['p_fc'].add_data(training_data['p_fc'], 'phosphosite', 'sample', 'value')

In [48]:
samples = list(set(training_data['e_ksea']['sample']).union(set(training_data['p_fc']['sample'])))
enzymes = list(set(training_data['e_ksea']['enzyme']))
evidence_dict = {}
for s in samples:
    evidence_dict[s] = {}
    for e in enzymes:
        evidence_dict[s][e] = {}
        phosphosites = data['p_regulates']['phosphosite'][data['p_regulates']['protein'] == e].tolist()
        for p in phosphosites:
            evid_generator_e = build.ProblogStatementGenerator(predicates['e_ksea'])
            evidence_e = evid_generator_e.generate_facts(build.EvidenceTemplate, select=[s, e])
            evid_generator_p = build.ProblogStatementGenerator(predicates['p_fc'])
            evidence_p = evid_generator_p.generate_facts(build.EvidenceTemplate, select=[s, p])
            evidence_dict[s][e][p] = evidence_p + evidence_e
        if len(phosphosites) == 0:
            evid_generator_e = build.ProblogStatementGenerator(predicates['e_ksea'])
            evidence_e = evid_generator_e.generate_facts(build.EvidenceTemplate, select=[s, e])
            evidence_dict[s][e][e] = evidence_e

In [45]:
# write evidence_dict to text file: loop over samples and separate by line of hyphens
with open('models/ebdt_data/sub_network/e_model/evidence_exp2.pl', 'w') as f:
    for s in evidence_dict:
        for e in evidence_dict[s]:
            for p in evidence_dict[s][e]:
                for fact in evidence_dict[s][e][p]:
                    f.write(fact + '\n')
                f.write('--------------------' + '\n')

In [49]:
# write evidence_dict to text file: loop over samples and separate by line of hyphens
with open('models/ebdt_data/sub_network/e_model/evidence_exp2_training.pl', 'w') as f:
    for s in evidence_dict:
        for e in evidence_dict[s]:
            for p in evidence_dict[s][e]:
                for fact in evidence_dict[s][e][p]:
                    f.write(fact + '\n')
                f.write('--------------------' + '\n')

LFI

In [5]:
DIR = 'models/ebdt_data/sub_network/e_model/'
max_iter = 1
# learning from interpretation
cmd = f'problog lfi {DIR}e_model_lfi_exp2.pl {DIR}evidence_exp2.pl -O {DIR}e_model_exp2_1i.pl --logger {DIR}log_exp2-1i.txt -k ddnnf -v -n {max_iter}'
os.system(cmd)

-2507.7641879654 [0.50577861, 0.49422139, 0.50401498, 0.49598502, 0.48814535, 0.51185465, 0.49711254, 0.50288746, 0.51781844, 0.60404647, 0.6198206, 0.33588868, 0.52248594, 0.2267053, 0.75274758, 0.99696768, 0.55731597, 0.7109984, 0.65227446, 0.39790327, 0.39601044, 0.62544722, 0.38498708, 0.57315608, 0.29424064, 0.45782059, 0.86146952, 0.77346495, 0.75780133, 0.69781716, 0.62540731, 0.61530862, 0.62335754, 0.33716673, 0.56627233, 0.31301228, 0.69646773, 0.95507724, 0.61418609, 0.83712526, 0.79847662, 0.34892854, 0.3792005, 0.62539979, 0.44231423, 0.57344463, 0.38393163, 0.34876666, 0.77966535, 0.64466411, 0.16937057, 0.15214016, 0.65824935, 0.59021679, 0.57404081, 0.55766146, 0.53292022, 0.63681195, 0.560153, 0.01565527, 0.38927609, 0.25589085, 0.63468415, 0.35404546, 0.3640534, 0.6251035, 0.48090358, 0.50758122, 0.55992616, 0.61153378, 0.74695416, 0.00643715, 0.77611688, 0.87186675, 0.57108837, 0.42237956, 0.5270773, 0.52348596, 0.38929847, 0.47234195, 0.78228101, 0.9909932, 0.620635

0

In [11]:
DIR = 'models/ebdt_data/sub_network/e_model/'
# learning from interpretation
cmd = f'problog ground {DIR}e_model_lfi_exp2.pl -o {DIR}e_model_exp2_ground.pl --format pl'
os.system(cmd)

(True, 't(0.33,"ABL1(S718)","AZD3759")::p_occupancy("ABL1(S718)","AZD3759",dec); t(0.33,"ABL1(S718)","AZD3759")::p_occupancy("ABL1(S718)","AZD3759",inc).\n0.2101821522::p_fc("ABL1(S718)","AZD3759",dec) :- p_occupancy("ABL1(S718)","AZD3759",dec).\n0.001::p_fc("ABL1(S718)","AZD3759",dec) :- p_occupancy("ABL1(S718)","AZD3759",inc).\n0.3949089239::p_fc("ABL1(S718)","AZD3759",dec).\n0.001::act_dec("ABL1","AZD3759").\n0.001::act_dec("ABL1","AZD3759").\n0.001::act_dec("ABL1","AZD3759").\nt(0.33,"ABL1(S569)","AZD3759")::p_occupancy("ABL1(S569)","AZD3759",dec); t(0.33,"ABL1(S569)","AZD3759")::p_occupancy("ABL1(S569)","AZD3759",inc).\nt(0.33)::p_function("ABL1(S718)",p_inc); t(0.33)::p_function("ABL1(S718)",p_dec).\nt(0.7,"ABL1(S569)","ABL1")::act_dec("ABL1","AZD3759") :- p_occupancy("ABL1(S569)","AZD3759",dec).\nt(0.7,"ABL1(S718)","ABL1")::act_dec("ABL1","AZD3759") :- p_function("ABL1(S718)",p_inc), p_occupancy("ABL1(S718)","AZD3759",dec).\nt(0.33,"ABL1(T735)","AZD3759")::p_occupancy("ABL1(T735

256